In [7]:
%pip install unidecode requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.12.14-py3-none-any.whl.metadata (2.3 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.12.14-py3-none-any.whl (164 kB)
Using cached charset_normalizer-3.4.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (145 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:

# Download cpdf from github
import platform, os, requests, subprocess

def get_system_info():
    os_name = platform.system()
    arch = platform.architecture()[0]
    machine = platform.machine()

    if os_name == "Linux":
        if "arm" in machine.lower() or "aarch" in machine.lower():
            return "Linux-ARM-64bit" if arch == "64bit" else "Linux-ARM-32bit"
        return "Linux-Intel-64bit" if arch == "64bit" else "Linux-Intel-32bit"
    
    elif os_name == "Darwin":  # macOS
        if "arm" in machine.lower() or "aarch" in machine.lower():
            return "OSX-ARM"
        return "OSX-Intel"
    
    elif os_name == "Windows":
        return "Windows64bit" if arch == "64bit" else "Windows32bit"
    

# Download cpdf
response = requests.get(f"https://github.com/coherentgraphics/cpdf-binaries/raw/refs/heads/master/{get_system_info()}/cpdf")
with open("cpdf", "wb") as f:
    f.write(response.content)
os.chmod("cpdf", 0o755)

subprocess.run(["./cpdf", "-help"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

CompletedProcess(args=['./cpdf', '-help'], returncode=0, stdout=b'', stderr=b'Syntax: cpdf [<operation>] <input files> [-o <output file>]\n\nCopyright Coherent Graphics Ltd.\n\nVersion AGPLv3-licensed 2.8. (10th December 2024)\n\nhttps://www.coherentpdf.com/\n\nInput names are distinguished by containing a \'.\' and may be\nfollowed by a page range specification, for instance "1,2,3"\nor "1-6,9-end" or "even" or "odd" or "reverse".\n\nOperations (See manual for full details):\n\n  -version                            Print the cpdf version number\n  -o                                  Set the output file, if appropriate\n  -i                                  Add an input file\n  -png                                Load from a PNG file, converting to PDF\n  -jpeg                               Load from a JPEG file, converting to PDF\n  -jpeg2000                           Load from a JPEG2000 file, converting to PDF\n  -jbig2                              Load from a JBIG2 fragment, conver

In [ ]:
import os, datetime
from unidecode import unidecode

input_folder = "corti"
output_folder = "corti-puliti"
cpdf_path = "./cpdf"


os.makedirs(output_folder, exist_ok=True)

def clean_name(original:str) -> str:
    return unidecode(original).replace(" ", "-").replace("/", "-").replace("\\", "-").replace(":", "-").replace("*", "-").replace("!", "").replace("?", "").replace("\"", "-").replace("<", "-").replace(">", "-").replace("|", "-")

def base_command(file:str) -> str:
    return " ".join(f"""{cpdf_path} "{input_folder}/{file}" -clean 
        AND -set-title "{file.replace(".pdf", "")} - Corti {datetime.datetime.now().year}"
        AND -set-author "Librogames's Land"
        AND -set-subject ""
        AND -set-keywords "librogame corti lgl" 
        AND -set-creator ""
        AND -set-producer "LGL Script by <luca.fabbian.1999@gmail.com>"
        AND -set-create "D:19970915110347-08’00’"
        AND -set-modify "D:19970915110347-08’00’"
        AND -set-untrapped
        AND -remove-metadata
        AND -set-language "it-IT"
        AND -scale-to-fit a5portrait
        -o "{output_folder}/{clean_name(file)}" """.split("\n"))

for file in os.listdir(input_folder):
    !{base_command(file)}
    print(file)

